In [4]:
# Librerias necesarias para procesamiento y entrenamiento
import mediapipe as mp
import cv2
import csv
import os
import numpy as np
import pandas as pd
import pickle
import pygame

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
mp_drawing = mp.solutions.drawing_utils   #Ayudas para dibujar
mp_holistic = mp.solutions.holistic       #Soluciones de Mediapipe

In [ ]:
# Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()      
        
        # Reformatear: BGR -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el modelo holistico:
        results = holistic.process(image)               
        
        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Mostramos la imagen                
        cv2.imshow(' ', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [7]:
results.face_landmarks.landmark[0]

x: 0.553945243
y: 0.522995174
z: -0.0399583764

In [8]:
print("x:", results.pose_landmarks.landmark[0].x,"y:", 
      results.pose_landmarks.landmark[0].y,"z:", 
      results.pose_landmarks.landmark[0].z)

x: 0.5199721455574036 y: 0.4256049692630768 z: -1.4726536273956299


In [9]:
#Tenemos 501 coordenadas en total
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [10]:
#la primera columna servirá de identificación de clase 
landmarks = ['class']

for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

#Mostramos ejemplo:
print(landmarks[0:13])

['class', 'x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', 'x3', 'y3', 'z3', 'v3']


In [11]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [12]:
class_name = "Cansado"

In [ ]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Detector - Cansado', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [14]:
class_name = "Despierto"

In [ ]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Detector - Despierto', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [16]:
# Leemos el archivo .csv
df = pd.read_csv("coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Cansado,0.463894,0.361057,-1.429761,0.999738,0.543382,0.322030,-1.411058,0.999411,0.575994,...,-0.012750,0.0,0.688389,0.377393,0.021292,0.0,0.695000,0.368503,0.022257,0.0
1,Cansado,0.479053,0.368109,-1.156163,0.999747,0.544634,0.321780,-1.124202,0.999438,0.576437,...,-0.011860,0.0,0.690490,0.370144,0.025524,0.0,0.696534,0.362520,0.026548,0.0
2,Cansado,0.501146,0.377613,-1.470222,0.999771,0.547348,0.329778,-1.425766,0.999492,0.579651,...,-0.013534,0.0,0.691514,0.369910,0.023277,0.0,0.697792,0.362378,0.024214,0.0
3,Cansado,0.520555,0.386492,-2.003457,0.999793,0.561482,0.337001,-1.945928,0.999537,0.596582,...,-0.014315,0.0,0.689669,0.370152,0.022396,0.0,0.695939,0.362208,0.023282,0.0
4,Cansado,0.549094,0.442984,-2.005894,0.999813,0.589027,0.369114,-1.945355,0.999582,0.624433,...,-0.016045,0.0,0.684296,0.375012,0.019446,0.0,0.690636,0.366923,0.020148,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Despierto,0.553653,0.460450,-1.782848,0.999735,0.601557,0.369811,-1.707001,0.999347,0.626466,...,-0.014824,0.0,0.678701,0.356397,0.021564,0.0,0.685422,0.349612,0.022063,0.0
250,Despierto,0.552506,0.461277,-1.768624,0.999735,0.599876,0.370056,-1.700109,0.999354,0.625291,...,-0.016101,0.0,0.679735,0.364270,0.019488,0.0,0.686804,0.355371,0.020171,0.0
251,Despierto,0.551497,0.462729,-1.705440,0.999744,0.599122,0.370440,-1.635473,0.999375,0.624700,...,-0.015229,0.0,0.680296,0.359665,0.019973,0.0,0.687277,0.351850,0.020447,0.0
252,Despierto,0.545311,0.462820,-1.621666,0.999742,0.595059,0.370342,-1.556187,0.999369,0.621303,...,-0.016995,0.0,0.685050,0.358752,0.018956,0.0,0.692120,0.350937,0.019439,0.0


In [17]:
# Guardaremos las variables en X y el target en Y
X = df.drop('class',axis=1)
y = df['class']

In [18]:
# Dividimos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1000)

In [19]:
# Definimos los cuatro pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [20]:
# Guardamos los modelos en la biblioteca fit_models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
# Probamos a predecir X_test con el modelo elegido (Random Forest)
fit_models['rf'].predict(X_test)

array(['Cansado', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Despierto',
       'Despierto', 'Cansado', 'Cansado', 'Despierto', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Despierto', 'Despierto',
       'Despierto', 'Despierto', 'Cansado', 'Cansado', 'Despierto',
       'Despierto', 'Cansado', 'Despierto', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Despierto', 'Cansado', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Cansado',
       'Cansado', 'Despierto', 'Cansado', 'Cansado', 'Despierto',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Despierto',
       'Despierto', 'Cansado', 'Despierto', 'Cansado', 'Despierto',
       'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Cansado', 'Cansado', 'Cansado', 'Despierto',
       'Cansado', 'Despierto', 'Despierto', 'Despierto', 'Despierto',
       'Cansado', 'Cansado', 'Despierto', 'Ca

In [22]:
#Predecir las clases de X_test
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    
    #Mostramos las precisiones de cada modelo
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [23]:
# Escribimos el modelo en un archivo .pkl
with open('cansado_despierto.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [24]:
# Abrimos de nuevo el modelo
with open('cansado_despierto.pkl', 'rb') as f:
    model = pickle.load(f)

In [25]:
# Inicializar pygame mixer
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound('alarma.wav')  # Asegúrate de que el archivo de sonido esté en formato .wav

In [26]:
# Conectamos la cámara
cap = cv2.VideoCapture(0)

# Inicializar la variable de control de la alarma
alarm_playing = False

# Inicia el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()

        # Verificar que la imagen se haya leído correctamente
        if not ret:
            print("No se pudo recibir imagen de la cámara. Saliendo...")
            break

        # BGR -> RGB y seguridad de la imagen
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detecciones
        results = holistic.process(image)

        # RGB -> BGR y seguridad de la imagen
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Dibujo de landmarks
        # 1. Face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )

        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

        # Exporta coordenadas
        try:
            # Verificar que se hayan detectado landmarks de pose y rostro
            if results.pose_landmarks and results.face_landmarks:
                # Extraemos Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([
                    [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose
                ]).flatten())

                # Extraemos Face landmarks
                face = results.face_landmarks.landmark
                face_row = list(np.array([
                    [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face
                ]).flatten())

                # Concatenamos las filas
                row = pose_row + face_row

                # Hacemos detecciones
                X = pd.DataFrame([row])

                # Predecir la clase
                body_language_class = model.predict(X)[0]

                # Predecir la probabilidad de la clase
                body_language_prob = model.predict_proba(X)[0]

                # Calcular la probabilidad
                probability = round(body_language_prob[np.argmax(body_language_prob)], 2)

                # Mostrar valores para depuración
                print(f"Clase predicha: '{body_language_class}'")
                print(f"Probabilidad: {probability}")

                # Recogemos coordenadas de la oreja izquierda para visualizar texto dinámicamente
                coords = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                    ), [640,480]).astype(int))

                # Mostramos la clase en la oreja izquierda
                cv2.rectangle(
                    image,
                    (coords[0], coords[1]+5),
                    (coords[0]+len(body_language_class)*20, coords[1]-30),
                    (245, 117, 16), -1)
                cv2.putText(
                    image, body_language_class, coords,
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Status Box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                # Mostramos Class
                cv2.putText(
                    image, 'CLASS', (95,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(
                    image, body_language_class.split(' ')[0], (90,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Mostramos Probabilidad
                cv2.putText(
                    image, 'PROB', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(
                    image, str(probability), (10,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Al estar muy cansado, activamos la alerta de descanso y la alarma
                if (probability > 0.70) and ("Cansado" in body_language_class):

                    # Mostrar mensaje de alerta
                    cv2.rectangle(
                        image,
                        (coords[0], coords[1]+5),
                        (coords[0]+len('MUY CANSADO')*20, coords[1]-30),
                        (0, 0, 255), -1)

                    cv2.putText(
                        image, 'MUY CANSADO', coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    cv2.rectangle(image, (0,0), (550, 60), (0, 0, 255), -1)

                    cv2.putText(
                        image, "ALERTA! NECESITAS DESCANSAR", (15,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    # Activar alarma si no está sonando
                    if not alarm_playing:
                        alarm_playing = True
                        alarm_sound.play(-1)  # Reproducir en bucle
                else:
                    if alarm_playing:
                        alarm_playing = False
                        alarm_sound.stop()
            else:
                print("No se detectaron landmarks de pose o rostro.")
                # Puedes manejar este caso si es necesario

        except Exception as e:
            print(f"Ocurrió un error en el procesamiento: {e}")
            pass

        # Mostrar la imagen
        cv2.imshow('DETECTOR DE SOMNOLENCIA', image)

        # Salir con la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Liberar la cámara y cerrar ventanas
    cap.release()
    cv2.destroyAllWindows()
    pygame.quit()

c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Despierto'
Probabilidad: 0.89


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Despierto'
Probabilidad: 0.89


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Clase predicha: 'Cansado'
Probabilidad: 0.9


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Cansado'
Probabilidad: 0.92


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Cansado'
Probabilidad: 0.96
